In [42]:
import json
import pandas as pd
from geopy.geocoders import Nominatim
from geografia_to_provincia import *
from geografia import dict_geo
from features import features_detected
from sklearn.preprocessing import LabelEncoder

In [3]:
with open('hackupc2023_restbai__dataset/hackupc2023_restbai__dataset_sample.json', encoding='utf-8') as file:
    # Load the JSON data
    data = json.load(file)

In [4]:
def extract_image_data_el(item, file):
    prop = file[item]
    image_data = prop["image_data"]
    # r1r6
    for el in ["property", "kitchen", "bathroom", "interior"]:
        prop["image_data.r1f6." + el] = image_data["r1r6"][el]

    # style
    for el in ["label", "confidence"]:
        prop["image_data.style" + el] = image_data["style"][el]
    
    
def extract_image_data(file):
    for item in file:
        extract_image_data_el(item, file)

In [13]:
extract_image_data(data)
d = data['303464']
d

{'summary': 'Dúplex en Les Arenes-Can Montllor-La Grípia',
 'city': 'Les Arenes-Can Montllor-La Grípia',
 'neighborhood': 'Distrito La Grípia-Sant Llorenç. Terrassa',
 'region': 'valles_occidental_nord',
 'price': 155000,
 'square_meters': 60,
 'bedrooms': 2,
 'bathrooms': 2,
 'images': ['https://restb-hackathon.s3.amazonaws.com/real_estate_dataset/images/303464__013.jpg'],
 'num_images': 1,
 'image_data': {'r1r6': {'property': None,
   'kitchen': None,
   'bathroom': None,
   'interior': None},
  'style': {'label': None, 'confidence': None},
  'features_by_room_type': {'terrace': {'unique_features': [{'label': 'patio',
      'details': []},
     {'label': 'washer__dryer', 'details': []}],
    'best_photo_idx': 0}}},
 'property_type': 'condo_apartment',
 'image_data.r1f6.property': None,
 'image_data.r1f6.kitchen': None,
 'image_data.r1f6.bathroom': None,
 'image_data.r1f6.interior': None,
 'image_data.stylelabel': None,
 'image_data.styleconfidence': None}

In [43]:
def dataset_from_json(d):
    df = pd.DataFrame.from_dict(d, orient="index")
    df = df.T
    for col in features_detected:
        df[col] = 0
    df = df.drop("image_data", axis=1)
    j = data['303464']['image_data']["features_by_room_type"]
    for k in j:
        for i in range (len(j[k]["unique_features"])):
            feature = j[k]["unique_features"][i]['label'];
            df[feature] = 1
    
    df['region'] = df['region'].str.replace('_', ' ')
    df['region'] = df['region'].str.replace('area de', '')
    df['region'] = df['region'].str.replace(' almeriense', '')
    df['region'] = df['region'].str.replace(' francesa', '')
    df['region'] = df['region'].str.replace(' frances iparralde', '')
    df['region'] = df['region'] + " españa"

#     print(df.region)
#     geolocator = Nominatim(user_agent="geoapiExercises")
#     address = geolocator.geocode(df['region'])
    
    address = dict_geo[df['region'][0]]
    list_provinces = return_province(address[0], address[1])

    for i in range(len(list_provinces)):
        if list_provinces[i] == True:
            df["id_prov_" + str(i) + ".0"] = 1
        else:
            df["id_prov_" + str(i) + ".0"] = 0
    
    house_json = {}
    columns_to_drop = ["summary", "region", "images", "city", "neighborhood"] 
    df = df.drop(columns_to_drop, axis=1)

    label_encoder = LabelEncoder()
    df['property_type_numeric'] = label_encoder.fit_transform(df['property_type'])

    for col in df.columns:
        house_json[col] = df[col][0]
    return house_json

In [44]:
house_json = dataset_from_json(d)

C:\Users\34665\AppData\Local\Temp\ipykernel_13096\3983626449.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
C:\Users\34665\AppData\Local\Temp\ipykernel_13096\3983626449.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[col] = 0
C:\Users\34665\AppData\Local\Temp\ipykernel_13096\3983626449.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. 

In [45]:
house_json

{'price': 155000,
 'square_meters': 60,
 'bedrooms': 2,
 'bathrooms': 2,
 'num_images': 1,
 'property_type': 'condo_apartment',
 'image_data.r1f6.property': None,
 'image_data.r1f6.kitchen': None,
 'image_data.r1f6.bathroom': None,
 'image_data.r1f6.interior': None,
 'image_data.stylelabel': None,
 'image_data.styleconfidence': None,
 'baseboard_radiator': 0,
 'central_ac': 0,
 'dishwasher': 0,
 'double_oven': 0,
 'microwave': 0,
 'oven': 0,
 'radiator': 0,
 'range_hood': 0,
 'range_oven': 0,
 'separate_washer_dryer': 0,
 'stacked_washerdryer': 0,
 'stovetop': 0,
 'refrigerator': 0,
 'tv': 0,
 'wall_mounted_ac': 0,
 'washer_dryer': 0,
 'water_heater': 0,
 'backsplash': 0,
 'bath': 0,
 'combined_bath_shower': 0,
 'double_sink': 0,
 'mirror': 0,
 'shower': 0,
 'sink': 0,
 'toilet': 0,
 'vanity': 0,
 'awning': 0,
 'balcony': 0,
 'basketball_court': 0,
 'bocce_ball': 0,
 'ceiling_fan': 0,
 'deck': 0,
 'dock': 0,
 'dog_park': 0,
 'dog_washing_station': 0,
 'fence': 0,
 'fire_pit': 0,
 'fire

In [220]:
most_important = ["image_data.r1f6.property", "pool", "vanity", "fireplace", "double_sink", "tv", "hardwood_floor", "mirror", "dishwasher", "floor_ceiling_windows", "microwave", "notable_chandelier", "pergola", "shower", "kitchen_island", "outdoor_bar", "oven", "radiator", "built_in_shelves","refrigerator"]
not_in = []
mi = {}
for col in most_important:
    mi[col] = 0

In [222]:
cat = ["pool", "vanity", "fireplace", "double_sink", "tv","hardwood_floor", "mirror", "dishwasher", "floor_ceiling_windows", "microwave", "notable_chandelier", "pergola", "shower", "kitchen_island", "outdoor_bar", "oven", "radiator", "built_in_shelves","refrigerator"]
for i in cat:
    if house_json[i] == mi[i]:
        not_in.append(i)

KeyError: 'pool'

In [202]:
not_in

[]